In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

import sys
from importlib import reload
from sklearn.metrics import confusion_matrix
import tensorflow as tf
#import keras
from scipy import stats
import scipy.signal as scisig
import scipy.stats
import dalmatian
from sklearn.metrics import f1_score, precision_score, recall_score
import seaborn as sns
#from tqdm.keras import TqdmCallback
from sklearn.model_selection import KFold
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score

from keras.layers import LSTM, Dense, Conv1D, TimeDistributed, Flatten, Activation, Dropout, Bidirectional,concatenate
from keras.callbacks import History, TensorBoard, Callback
import keras.initializers as KI
from keras.layers import BatchNormalization

from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import tensorflow_addons as tfa
from tqdm.keras import TqdmCallback

In [7]:
conf_matrix_list_of_arrays = []
scores = []
acc_per_fold = []
loss_per_fold = []
f1_per_fold = []
prec_per_fold = []
rec_per_fold = []

participants = ["p" + str(i).zfill(2) for i in range(1, 16) if i != 3 and i != 4]
output_path = "../../data/output/E4_comb_mods/"

def s(samples):
    std = np.std(samples)
    if std == 0:
        return samples - np.mean(samples)
    else:
        return (samples - np.mean(samples)) / std

acc_freq = 32
eda_freq = 4
temp_freq = 4
bvp_freq = 64
hr_freq = 1

#sliding window - set window size and step size 
def create_window(pid, win_size=30, win_step=15, Binary=False):
    
    for f in os.listdir(output_path):
        if f.startswith(str(pid)) and f.endswith("ACC.csv"):
            acc_path = f
        elif f.startswith(str(pid)) and f.endswith("EDA.csv"):
            eda_path = f
        elif f.startswith(str(pid)) and f.endswith("TEMP.csv"):
            temp_path = f
        elif f.startswith(str(pid)) and f.endswith("BVP.csv"):
            bvp_path = f
        elif f.startswith(str(pid)) and f.endswith("HR.csv"):
            hr_path = f
    
    acc_data = pd.read_csv(output_path + acc_path, index_col=0 )
    eda_data = pd.read_csv(output_path + eda_path, index_col=0)
    temp_data = pd.read_csv(output_path + temp_path, index_col=0)
    bvp_data = pd.read_csv(output_path + bvp_path, index_col=0)
    #hr_data = pd.read_csv(output_path + hr_path, index_col=0)

    eda_data['interest'] = eda_data['interest'].replace([1,2,3,4], [0,1,2,3])
    bvp_data['understand'] = bvp_data['understand'].replace([1,2,3,4], [0,1,2,3]) 
    
    acc_x = []; acc_y = []; acc_z = []; eda = []; temp=[]; label=[]; bvp=[]; hr=[]; ut=[]; win_size = 30 ; win_step= 15; 
    total_time = int(len(eda_data) / eda_freq)

    for i in range(win_size, total_time, win_step):

        acc_x.append(s(acc_data["acc_x"][acc_freq * (i - win_size): acc_freq * i]))
        acc_y.append(s(acc_data["acc_y"][acc_freq * (i - win_size): acc_freq * i]))
        acc_z.append(s(acc_data["acc_z"][acc_freq * (i - win_size): acc_freq * i]))

        eda.append(s(eda_data["data"][eda_freq * (i - win_size): eda_freq * i]))
        temp.append(s(temp_data["data"][temp_freq * (i - win_size): temp_freq * i]))
        bvp.append(s(bvp_data["data"][bvp_freq * (i - win_size): bvp_freq * i]))
        #hr.append(s(hr_data["data"][hr_freq * (i - win_size): hr_freq * i]))
        ut.append(bvp_data["understand"][bvp_freq * (i - win_size): bvp_freq * i])

        l = stats.mode(eda_data["interest"][eda_freq * (i - win_size): eda_freq * i])[0][0]
        if Binary:
            if(l == 0) | (l == 1):
                l = 0
                label.append(l)
            else:
                l = 1
                label.append(l)
        else:

        #l_vote = np.bincount(np.array(l)).argmax()
            label.append(l)


    return {
        "label": np.array(label),
        "acc_x": np.array(acc_x), 
        "acc_y": np.array(acc_y), 
        "acc_z": np.array(acc_z),
        "eda": np.array(eda),
         "bvp": np.array(bvp), 
         "temp": np.array(temp),
         "ut": np.array(ut)}

In [8]:
def train(train_dataset, test_dataset):
    models = []
    for feature in ["acc_x", "acc_y", "acc_z", "eda", "bvp", "temp" ,"ut"]:
    #for feature in [ "bvp", "eda", "temp", "ut"]:

        input_size = train_dataset[feature].shape[1]
        #filter_size = int(input_size / 10)  # hyper param
        #filter_size = 3
        input_layer = tf.keras.layers.Input(shape=(input_size, 1))
        conv_layer_1 = tf.keras.layers.Conv1D(
            50, 3, activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(1e-3))(input_layer)
        conv_layer_2 = tf.keras.layers.Conv1D(
            30, 3, activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(1e-3))(conv_layer_1)
        conv_layer_3 = tf.keras.layers.Conv1D(
            10, 3, activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(1e-3))(conv_layer_2)
        pooling_layer = tf.keras.layers.GlobalAveragePooling1D()(conv_layer_3)
        models.append(tf.keras.Model(inputs=input_layer, outputs=pooling_layer))

    concat_layer = tf.keras.layers.concatenate([x.output for x in models])
    dense_layer_1 = tf.keras.layers.Dense(128, activation="relu")(concat_layer)
    dense_layer_2 = tf.keras.layers.Dense(64, activation="relu")(concat_layer)
    output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(dense_layer_2)

    model = tf.keras.Model(inputs=[x.input for x in models], outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics=['accuracy'])

    X_train = []; scalers = {}
    for feature in ["acc_x", "acc_y", "acc_z", "eda", "bvp", "temp" ,"ut"]:
        X_train.append(np.reshape(train_dataset[feature], (-1, train_dataset[feature].shape[1], 1)))
    y_train = train_dataset["label"]

    X_test = []
    for feature in ["acc_x", "acc_y", "acc_z", "eda", "bvp", "temp" ,"ut"]:
        X_test.append(np.reshape(test_dataset[feature], (-1, test_dataset[feature].shape[1], 1)))
    y_test = test_dataset["label"]


    history = model.fit(X_train, y_train, epochs=10, batch_size=512, validation_split=0.2, verbose=1)
    scores = model.evaluate(X_test, y_test)
    print(f'Score : {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1])
    loss_per_fold.append(scores[0])
    
    
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    f1_per_fold.append(f1_score(y_test, y_pred, average='weighted'))
    prec_per_fold.append(precision_score(y_test, y_pred, average='weighted'))
    rec_per_fold.append(recall_score(y_test, y_pred, average='weighted'))
    print('F1 score : ', f1_score(y_test, y_pred, average='weighted'))


    cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    print('Confusion Matrix : ')
    print(cm)
    
    conf_matrix_list_of_arrays.append(cm)
    return conf_matrix_list_of_arrays, acc_per_fold, f1_per_fold

In [9]:
comb_data = [create_window(x) for x in participants]

In [133]:

per_acc = []; per_f1 = []; per_pred = []; per_lab = []

X = {k:v for k,v in comb_data[0].items() if k in ["acc_x", "acc_y", "acc_z", "eda", "bvp", "temp" ,"ut"]}
#X = list(X.values())
y = comb_data[0]["label"]

kfold = KFold(n_splits=5, shuffle=True, random_state=49)
fold_no = 1

for train_index, test_index in kfold.split(X['bvp'],y):

    #train(X["bvp"][train_index], y[test_index])

    


    print (train_index)
    #print (y[test_index])
    

#     per_pred.extend(preds)
#     per_lab.extend(y_test)

# per_acc.append(np.mean(acc))
# per_f1.append(np.mean(f1))

[  0   1   3   4   5   8   9  10  11  12  14  15  16  18  19  20  22  24
  25  26  27  28  30  31  32  33  34  35  36  37  39  40  41  42  43  44
  47  49  50  51  52  54  56  57  58  59  60  61  62  63  64  66  71  72
  73  74  75  76  77  78  79  80  82  83  85  86  87  88  89  91  92  94
  95  96  97  98  99 100 101 103 104 105 106 107 109 111 112 113 114 116
 117 118 119 120 121 122 123 124 125 126 127 129 130 132 133 134 135 136
 137 138 140 141 142 143 144 145 146 147 148 151 153 154 155 156 157 158
 160 161 162 163 164 166 167 168 169 170 171 172 173 174 175 177 178 179
 180 181 182 183 185 186 188 189 190 192 193 194 195 197 198 199 200 201
 203 204 205]
[  2   3   4   5   6   7  10  12  13  14  16  17  20  21  22  23  25  27
  28  29  31  32  33  34  35  36  37  38  39  40  41  43  45  46  47  48
  49  50  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68
  69  70  71  72  73  75  77  78  79  80  81  82  84  85  86  88  89  90
  91  92  93  94  95  97  98  99 100 

In [120]:
vals = np.array(list(X.values()), dtype=object)


ValueError: could not broadcast input array from shape (207,960) into shape (207,)

In [135]:
X['acc_x'].shape

(207, 960)